# Masterthesis
## Shap and Shapley-Values

**Imports and Definitions**
- The necessary libraries are loaded here and important variables are defined

**Imports and settings for this script**
- Import libraries and set variables for this script

**Create different plots to find the best presentaion for this use case**
- This step create diffent plots to find the best use case for SHAP-Values. 

  - Feature Importance Plot
  - Waterfall Plot
  - Bar Plot
  - Line Plot

**Create Heatmap**
- A heat map is created to display the SHAP values of the individual scenarios. This is the best use case for displaying the SHAP values.

**Create plot for shap summary**
- This plot shows the Subcarrier with the highest scores


## Imports and Definitions

In [ ]:
# Import sklearn
import sklearn

# Import pandas
import pandas as pd

# Import numpy
import numpy as np

# To calculate amplitude and phase
import math

# Measure runtime of a jupyter jotebook code cell
from timeit import default_timer as timer

# Used to check if file exists
import os

# Used to check if directory exists
import pathlib

# Import Operation System Calls
import SubOperationSystem

# check os
if os.name == 'nt':
    print("OS is Windows")
    Delimiter = '\\'
    
else:
    print("OS is Linux")
    Delimiter = '/'
    
# Path of datasets (root directory)
PathDataset = 'Dataset' + Delimiter    

# Path of datasets
PathDatasetSub = PathDataset + 'CsiFilesRah' + Delimiter
        
# Path of the converted files
PathConverted = PathDataset + 'Converted' + Delimiter

# Set path for scenario files
PathScenario = PathDataset + 'Scenario' + Delimiter

# Set path for scenario files
PathResult = PathDataset + 'Result' + Delimiter

# Set path for scenario files
PathPlot = PathDataset + 'Plot' + Delimiter

# Set path for scenario files
PathConfig = 'FilesConfig' + Delimiter

# Scenariofile (file with info about the ten scenarios)
FileScenario = 'FileScenario.csv'

# Mappingfile (file with info about original and converted filenames)
FileMapping = 'FileMapping.csv'

## Imports and settings for this script

In [ ]:
# import
from sklearn.model_selection import train_test_split

# Import xgboost
import xgboost as xgb

# Import shap
import shap

# Import matplotlib
import matplotlib.pyplot as plt

# Import Metrics
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

# Set random_state
random_state = 42

# Extension used for filenames
FileExtension = '_xgb'

# Set name for scenario result file
ScenarioResultFile = 'scenarioResults.npy'

## Create different plots to find the best presentaion for this use case

In [ ]:
# Read scenario mapping file
dfScenario = pd.read_csv(PathConfig + FileScenario, sep=',')

# Create array for scenario results
scenarioResults = np.zeros((0, 52))

# Loop through scenario file
for ind in dfScenario.index:
    
    # Get filename of scenario
    Scenario = (dfScenario['Scenario'][ind])
                    
    # Read scenario file
    df = pd.read_csv(PathScenario + Scenario + '_ah.csv')

    # Get samples without label
    X = df.drop(columns=['label'])

    # xgboost needs labels from 0 to ... n
    # ----------------------------------------------------------------------------------------
    
    # First: get names of unique classes and save it into classes
    classes = df['label'].unique()
    
    # Second: create dictionary with original y values
    classesDictionary = {i: value for i, value in enumerate(classes)}

    # Third: replace y-value with values beginning from 0 to ...
    for key, value in classesDictionary.items():
        df.loc[df['label'] == value, 'label'] = key

    # Get column label
    y = df['label']

    # Split in train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

    # Get columnnames of x
    features = list(X)
    
    # Get only the number of features for legends in plot
    featureNames = [s.replace('csi_subcarrier_', '') for s in features]

    # Clean up the Scenario name for plot
    scenarioName = Scenario.replace('Scenario','Szenario ')
    scenarioName = scenarioName.replace('10','zz')
    scenarioName = scenarioName.replace('0','')
    scenarioName = scenarioName.replace('zz','10')
        
    # Fit XGBoost-Modell
    model = xgb.XGBClassifier(objective="multi:softprob", num_class=len(classes))
    # model = xgb.XGBClassifier(objective="multi:softprob", num_class=len(classes, eval_metric='mlogloss'))
    
    # fit model
    model.fit(X_train, y_train)

    # Get and print accuracy
    y_pred = model.predict(X_test)
    
    #############################################################################################
    # Classification

    # Create classification for accuracy for each label
    reportLabel = classification_report(y_test, y_pred, output_dict=True)

    # Extract accuracy for each label
    accuracy_per_class = {str(int(cls)): metrics['precision'] for cls, metrics in reportLabel.items() if cls.isdigit()}

    # Convert report to pandas format and save accuracy acc file
    acc_report = pd.DataFrame([accuracy_per_class]).transpose()
    acc_report.to_csv(PathResult + Scenario + FileExtension + '.acc', index=True)

    # Create report for classifikation
    reportClassification = classification_report(y_test, y_pred)

    # Create F1-Score report
    reportF1Score = accuracy_score(y_test, y_pred)

    # Print scenario name
    print('Scenarioname:', scenarioName)
    print(80 * '-')
    
    # Print Classification report
    print(f'\nClassification Report:\n{reportClassification}')
    print(f'\nF1-Score: {reportF1Score}')

    # Convert report to pandas format
    clsf_report = pd.DataFrame(classification_report(y_true = y_test, y_pred = y_pred, output_dict=True)).transpose()

    # Save report to file
    clsf_report.to_csv(PathResult + Scenario + FileExtension + '.result', index=True)

    #############################################################################################
    # Create explainer and calculate shap values
    
    # explainer = shap.Explainer(model)
    explainer = shap.TreeExplainer(model)

    # Get shap values (needed )
    # more information as only shap-values (new version)
    shap_values = explainer(X_test)
    # return value is only a numpy with shap-values (old version)
    # shap_values_shap = explainer.shap_values(X_test)

    # Get shap_values to array
    shap_values_array = shap_values.values
    
    # Sum the average absolute SHAP values across all classes
    mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=(0, 2))
    
    #############################################################################################
    # Create Feature Importance plot
    
    # Define figure size
    plt.figure(figsize=(10, 15))
    
    # Set bar plot
    plt.barh(range(len(mean_abs_shap_values)), mean_abs_shap_values, color='lightblue')
    
    # Set ysticks
    plt.yticks(range(0,len(featureNames)), featureNames)

    # Set xlabel
    plt.xlabel('Mean |SHAP Value|')
    
    # Set title of plot
    plt.title('Mean Absolute SHAP Values Summed Over All Classes')
    
    # Save plot
    plt.savefig(PathPlot + Scenario + FileExtension + '_fi-plot.png')
    
    # Show plot
    plt.show() 

    #############################################################################################
    # Calculate mean shap values
    mean_abs_shap_values_per_class = np.mean(np.abs(shap_values_array), axis=0)
    
    # Transpose values
    mean_abs_shap_values_per_class = mean_abs_shap_values_per_class.T

    # Create a emty array 
    myArray = np.zeros((0, 3))

    for classesKey, classesValue in classesDictionary.items():
        topIndizes = np.argsort(-mean_abs_shap_values_per_class[classesKey])[:8]
        topVales = mean_abs_shap_values_per_class[classesKey, topIndizes]
    
        count = 0

        for aa in topIndizes:
            new_row = np.array([classesDictionary[classesKey], featureNames[aa], topVales[count]])
            myArray = np.append(myArray, [new_row], axis=0)
            
            # Increase count
            count += 1

    #############################################################################################
    # Save results
    
    np.savetxt(PathResult + Scenario + FileExtension + '_shap.csv', 
               myArray, 
               delimiter=',', 
               comments='', 
               fmt='%s')
    
    np.savetxt(PathResult + Scenario + FileExtension + '_mean.csv', 
               mean_abs_shap_values_per_class, 
               delimiter=',', 
               header='Class,Feature,Value', 
               comments='', 
               fmt='%s')

    # Append scenario results to scenarioResults for later use
    scenarioResults = np.append(scenarioResults, mean_abs_shap_values_per_class, axis=0)

    #############################################################################################
    # Create Waterfall plot
    
    shap_values_index = 0
    instance_index = 0
    expected_value_index = 0
    
    shap.initjs()
    
    # Create waterfall plot
    shap.waterfall_plot(shap.Explanation(
            values=shap_values[shap_values_index][:, instance_index], 
            base_values=explainer.expected_value[expected_value_index], 
            feature_names=features),
            max_display=8, show=False)
    
    # Save plot as png file
    plt.savefig(PathPlot + Scenario + FileExtension + '_wf-plot.png', bbox_inches='tight')

    #############################################################################################
    # Create bar plot

    # Counts of classes and features
    n_classes, n_features = mean_abs_shap_values_per_class.shape

    # Bar width
    bar_width = 0.2

    # Positions of bars
    positions = np.arange(n_features)

    # Create bar plot
    plt.figure(figsize=(15, 6))
    for i in range(n_classes):
        plt.bar(positions, 
                mean_abs_shap_values_per_class[i], 
                width=bar_width, 
                label='Klasse: ' + str(classesDictionary[i]),
                bottom=np.sum(mean_abs_shap_values_per_class[:i], 
                axis=0)
               )

    # Set x label
    plt.xlabel('Features')
    
    # Set y label
    plt.ylabel('Mean Abs SHAP Values')

    # Set xticks
    plt.xticks(range(0,len(featureNames)), featureNames)
    
    # Set title
    plt.title(scenarioName)
    
    # Set legends with two columns
    plt.legend(ncol=2)
    
    # Tight layout
    plt.tight_layout()
    
    # Save plot to filesystem
    plt.savefig(PathPlot + Scenario + FileExtension + '_b-plot.png')
    
    # Show plot
    plt.show()

    #############################################################################################
    # Create Line plot
    
    # Set size of diagram
    plt.figure(figsize=(15, 6))

    # Create diagram
    for i in range(n_classes):
        classname = ('Klasse: ' + str(classesDictionary[i]))
        plt.plot(mean_abs_shap_values_per_class[i], label=classname)

    # Set xlabel
    plt.xlabel('Features')
    
    # Set ylabel
    plt.ylabel('Mean Abs SHAP Values')
    
    # Set title
    plt.title('Mean Abs SHAP Values pro Klasse')
    
    # Set xsticks
    plt.xticks(range(0,len(featureNames)), featureNames)
    
    # Set legend to two columns
    plt.legend(ncol=2)
    
    # Set layout
    plt.tight_layout()

    # Save plot as png
    plt.savefig(PathPlot + Scenario + FileExtension + '_l-plot.png')

    # Show plot
    plt.show()
    
# save result as numpy for later use
np.save(ScenarioResultFile, scenarioResults)                


## Create Heatmap

In [ ]:
# features for y-axis
featuresValues = [6,8,10,12,14,16,18,20,22,24,26,28,30,31,33,34,36,38,40,42,44,46,48,50,52,54,56,58]

# positon of features on the y-axis
featuresPosition = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,26,27,29,31,33,35,37,39,41,43,45,47,49,51]

# Result data ranges from Scenarion 1 to 10 (Scenario01=0to8, Secenario02=8to16 and so on)
listDataResult= [0,8,16,24,27,42,60,65,70,75,85]

# load saveed result als numpy file
scenarioResults = np.load(PathDataset + ScenarioResultFile)

# open scenario mapping file to read
dfScenario = pd.read_csv(PathConfig + FileScenario, index_col=0)

# Set counter
counter = 0

# loop through scenario dataframe
for ind in dfScenario.index:
    
    # get scenarios and dataset from dataframe
    Scenario,Datasets = (dfScenario['Scenario'][ind], dfScenario['Datasets'][ind])
    
    # split to dataset items because we need a int value
    DatasetItems=list(Datasets.split())
    DatasetItemsLen = (len(DatasetItems))
   
    # Convert str to int
    DatasetItems = list(map(int, DatasetItems))
    
    # get data
    npScenario = scenarioResults[listDataResult[counter] : listDataResult[counter+1]]
    
    # transponiere
    npScenario = npScenario.T

    # get row and columns
    rows, columns = npScenario.shape

    # Get classes
    classes = range(columns)

    # Set plot size - x length
    if counter == 4:
        xLength = 3.6
    elif counter == 5:
        xLength = 4
    else:
        xLength = 3.1
    
    # Create plot
    fig, ax = plt.subplots(figsize=(xLength, 8))

    # Create a picture based on npScenario
    im = ax.imshow(npScenario)

    # set x-axis
    x_ticks = np.arange(len(classes))

    # ax.set_xticks(x_ticks[::2])
    ax.set_xticks(range(DatasetItemsLen),DatasetItems)

    # set y-axis
    ax.set_yticks(featuresPosition, labels=featuresValues)
    
    # Rotate the tick labels
    plt.setp(ax.get_xticklabels(),rotation=90)

    # Add colorbar 
    cbar = ax.figure.colorbar(im, ax = ax )

    # Set xlabel
    plt.xlabel("Klassen")

    # Set ylabel
    plt.ylabel("Merkmale (Subträger)")

    # Increase counter
    counter +=1
    
    # Set title of plot
    ax.set_title("Szenario " + str(counter) + " - SHAP-Werte")
    
    # set layout
    fig.tight_layout()
    
    # Save plot as png
    plt.savefig(PathPlot + Scenario + '_Heatmap.png')
    
    # Show plot
    plt.show()
    
       
    
    

## Create SHAP summary plot

In [ ]:
# Data to plot
X = np.array(["6","58","33","57","40","31","34","47","43","46","42","47","48"])
y = np.array([8,6,5,4,4,3,2,2,2,2,2,2,2])

# Create plots
plt.figure(figsize=(10, 6))
plt.bar(X,y)
plt.xlabel('Bezeichung der Subträger')
plt.ylabel('Anzahl der Subträger')
plt.title('Übersicht der Subträger mit den höchsten Scores')
plt.savefig(PathPlot + 'SummaryPlot-SubcarrierShap.png')
plt.show()